In [1]:
from collections import namedtuple

In [6]:
Token = namedtuple("Token", ["token_name", "attribute_value"])
# token_name 名字 可以是任意指定字符串，也可以直接就是词素，如 10, +, -, *, /, (, ), =,
# 词法单元的名字是词法分析器进行语法分析时使用的抽象符号
# attribute_value 属性值 可以不存在

## 语法

- 我们使用上下文无关文法用来组织编译器前端。后称，文法
- `stmt -> if (exp) stmt else stmt`
  - `->` 具有如下形式
  - 这样的规则是产生式（`production`）
  - 在产生式中，`if` 和括号被称为终结符号（`terminal`）
  - `expr` 和 `stmt` 是终结符号的序列，为 `nonterminal` 非终结符号
- 文法 CFG 组成
  - 一个终结符号的集合，也被称为词法单元
  - 一个非终结符号的集合，成为语法变量。非终结符号和终结符号的集合（即，终结符号是字母表）
  - 一个产生式的集合
    - 产生式头，即最左边的非终结符号
    - 一个箭头
    - 一个产生式体，即由右边的非终结符号以及终结符号组成的序列
      - 以同一个非终结符号为头部的产生式体放在一起表示，用 `|` 分割
- 词法分析器
  - 词法分析器把字符序列组织为由词法含义的词素，生成并输出这些词素的词法单元序列（也就是，`Token`)
  - 此法单元的名字一般被成为终结符号
  - 属性值是指向符号表的指针，包含了该词法单元的附加信息。这些信息不是文法的组成部分。

### 例子

一个简单的加减乘除计算器

*list*->*list*+*digit*|*list*-*digit*|*digit*

*digit*->0|1|2|3|4|5|6|7|8|9

- 上述文法描述了一个由 `+`, `-`, `*`, `/` 分割的数位序列，或表达式
- 我们说，这样的文法
  - 包含终结符号 `+-*/0123456789`
  - *list* 和 *digit* 是非终结符号。特别的，因为 *list* 被首先列出，还作为这个文法的开始符号
  - 非终结符号由终结符号串组成，每个串有 0 - 无限个终结符号。特别的，0 个终结符好串为空串。

方法调用

*call* -> id( *optarams* )
*optparams* -> params | $e$
*params* -> *params*,*params* | *params*

## 使用

我们从开始符号出发，递归的将非终结符号替换为终结符符号的某个产生式的体。所有可在这个推
导过程中得到的终结字符串的集合，为这个文法定义的语言。

### 例子

我们说，9-5+2 是可以从上述文法推出的语言

- 9 是 `list`
- 9 -5 是 `list`
- 9-5 +2 是 `list`

## 词素

很多时候，单个字符的词素是不够的。不过，这个问题我们留到以后解决。

## 语法分析树

语法分析是所有编译过程中最基本的问题之一。

![语法分析树](images/ea42bfb2b39b58254c8662f84b1ff5a3e0ea3ab341d58dcdb946f2c8c513aa6a.png)  

parse tree 是一个树，有如下特点
- 根节点是文法的开始符号
- 每个叶子节点为 $e$ 或者终结符号
- 所有非叶子节点都是非终结符号（又称为内部节点 interior node）
- 如果非终结符号 $A$ 是某个内部节点的标号，且其子节点从左往右为 $X_1, X_2 \ldots X_n$，
  那么必须存在 $A \rightarrow X_1X_2\ldots X_n$。特别的，如果 $A->e$ 是一个表达式，他也
  可以只有一个标号为 $e$ 的子节点。
- 我们要求兄弟节点保持有序关系。

语法分析，**就是构建语法分析树**

### 二义性

仍然考虑我们之前的表达式。我们发现，这个表达式保证了计算是从左至右计算的。试考虑如下定
义

*str* -> *str*+*str*|*str*-*str*|0|1|2|3|4|5|6|7|8|9

这个定义不靠谱，比如， 9 - 5 + 2 这样的表达式，这个定义会产生了两棵树。

![二义性](images/d427e20005610741beb9dba51790a23c5f011da5aa544663b22ebf6f20f70016.png)  

然后我们之前定义的语法将会保证只有从左到右的计算。我们将其归类，这样的写法 *x*->*x*+*y*
都是左结合的，例如 `+ - * /`。例子：1+2+3，一个运算分量左右两侧都有加号时，属于左边的，
即(1+2)+3。

右结合，*right*->*id*=*right*|id。这就意味着，如果一个运算分量两边都有运算符，属于右
边。比如，赋值运算 `a=b=c -> a=(b=c)`。

### 优先级

如果加入加减乘除，就还要考虑优先级的问题。例如 `*` 比 `-` 有更高的优先级。
为了解决这个问题，我们把他们分开来创建生成式。

- 加减 *expr* -> *expr* + *term* | *expr* - *term* | *term*
- 乘除 *term* -> *term* \* *factor* | *term* / *factor* | *factor*
- 表达式的基本单元 *factor* -> *digit* | (*expr*)

可以这样理解：factor 是不能被任何运算符分开的表达式，term 可以被 \* 或者 / 分开，expr可
以被任何运算符分开。推广来说，如果希望 $n$ 层优先级，使用 $n+1$ 个非终结符号即可。每个
优先级都有一个非终结符号，表示能被该优先级或者更高优先级分开的表达式，然后就可以获得一
堆产生式体来表示。这个非终结符号的意义在于，长生那些只包含更高优先级的非终结符号。

## 练习

S -> S S + | S S * | a

1. aa+a*
- 所有的 a 都被 S 匹配，最后一个分支
- aa+ 是 S 被一个匹配
- aa+a* 是 S 被第二个匹配

![生成树](images/ecc0072b26bf800d0cf8492bb4eb87674313be02402389616aa36704566f817c.png)  

- 该文法生成以 + 或者 * 结尾的后缀表达式

2. 生成语言
- 生成 00...11 嵌套
- 生成以 a 作为变量的，包含 + - 的前缀表达式
- 生成任意层数的嵌套的括号，如 (()(()))
- 生成成对的 ab ba 特别的，他们可以插入在 ab/ba 中间或者后面
- 生成算术表达式，特别的，两个运算分量之间可以没有运算符，或者有一个到无数个*结尾

3. 5 有，如 a+a+a 可以是 (a+a)+a 也可以是 a+(a+a). 4 有 abab 可以是 a ba b 也可以是 ab
   ab. 3. 有， ()() 左边的 S 匹配，也可以是 右边的 S 匹配了 ()

4. exp -> exp exp op | num
   
   list -> list, id | id
   
   list -> id, list | id

   expr -> expr + term | expr - term | term
   term -> term * term | term / term | factor
   factor -> id | num | (expr)

   expr -> expr + term | expr - term | term
   term -> term * term | term / term | u
   u -> - factor | + factor | factor // I believe + u should also work. I shall
   implement that so you can write - - -1 = -1
   factor -> id | num | (expr)

   

## 语法制导翻译

语法制导翻译 (syntax-directed translation) 是一种实现编译器的思路。

其核心思想是围绕语法定义，通过解析语法来驱动整个编译的完成。

语法制导翻译可以通过向语法符号附加规则､定义､需要执行的代码（称为动作 (Action)）等信息来实现。

### 两个概念

- 属性 Attribute ：属性表示和某个程序构造相关的任意的量，可以是表达式的数据类型、生成的代码中的指令数目、或为某个构造生成代码中第一条指令的位置。
- 翻译方案 Translation scheme: 将程序片段附加到文法的产生式上的方法。这样，每次使用一个产生式，相应的程序片段就会执行。把执行结果拼接起来，就得到的翻译结果。

## 例子（后缀表示）

一个表达式 E 的 postfix notation 被这样定义
- E 是一个常量或者变量，E 的后缀表示是 E 本身
- E 是一个形如 E1 op E2 的表达式，op 为二元运算符，那么 E1'E2'op 为其后缀表达式，
  E1'E2' 为E1E2 后缀表达式的形式
- 如果 E 为 (E) 那么后缀表达式 E' = (E)'

### 综合属性

给文法的每个产生式附加语义规则。对于语法分析树的每一个节点，如果其关系是某个产生式，那
么该产生式对应的规则就描述了如何计算这个节点上的属性。

**语法制导定义 (syntax-directed definition)**
- 把每个文法符号和一个属性集合相关联
- 把每个产生式和一组语义规则相关联 semantic rule

对于节点 N，其文法符号为 X (token name)
- 用 X.a 表示其属性 a 的值。

一个有属性值的语法分析树是注释的语法分析树。如果一个属性尤其子节点定义，则为
synthesized atrribute （对树进行深度优先遍历即可）

继承属性，也就是由父节点决定。

![后缀表达式树](images/72549c1948c01ce9d3145338bef9c87b748191ae0fe2a023ad8d5e0808cb523b.png)  

### 后缀转化

对产生式 *expr* -> *expr1* + *term* 加入语义规则， *expr.t* = *expr1.t* || *term.t* ||
'+' 就可以获得后缀表达式。
- 我们使用 1,2,3 来表示同一个非终结符号在自身产生式中反复出现的情况。

### 简单语法制导

如果需要得到非终结符号结果的内容，只需要将其内部非终结符号的翻译结果或属性，加入一些字
符串连接起来即可，那么这就是**simple** 语法制导定义。

### 翻译方案

语义动作，semantic action，把一个语义动作用花括号括起来，并写入产生式体中即可。这部分代
码会执行用于翻译，如。

*rest* -> + *term* {print('+')} *rest_1*
- 特别的，这里的 *rest* 代码一个表达式中除了第一个项之外的一切。

![图 5](images/e6b3f0fa966389e8ee62767ed4fd79d0bf60ce9a714ff1001233c25a5861d397.png)  

深度优先递归一边，每个都执行一下，就可以得到后缀表达式形式。



## 递归下降分析器

- 一次性从左到右扫描输入
- 每次向前看一个终结符符号
- 成功则前进，失败则回溯尝试别的。最终，构造出分析树的各个部分或者报错。

大部分语法分析可归入以下两类

- 自顶向下
- 自底向上
- 这主要指代语法分析树节点构造顺序。自顶向下，构造过程从根节点开始，逐步走向叶子节点。
  自顶向下的分析器更受欢迎，因为比较容易的构造高效的分析器。

### 例子

生成 C/Java 语句的一个子集。

*stmt* -> **expr**; | **if**(**expr**) *stmt* | **for**(*optexpr*;*optexpr*;*optexpr*)
*stmt* | **other**

*optexpr* -> $e$ | **expr**

In [31]:
import regex as re

TERMINAL_SYMBOLS = re.compile(
    # 忽略空白符
    r"\s*(\L<terminal_symbols>)\s*",
    terminal_symbols=["for", "if", "other", "(", ")", ";", "expr"],
)

input_string = "for(;expr;expr)if(expr)other"

def lexer(input_string):
    """
    分词器
    """
    return [lexeme for lexeme in TERMINAL_SYMBOLS.split(input_string) if lexeme]

lexemes = lexer(input_string)
idx = 0

def match(root, *exp_lexemes):
    global idx
    for lexeme in exp_lexemes:
        if(lexemes[idx] == lexeme and idx < len(lexemes)):
            root.children.append(Node(Token(lexeme, None), []))
            idx += 1
        else:
            raise ValueError("Expected %s, got %s" % (lexeme, lexemes[idx]))

Node = namedtuple("Node", ["token", "children"])

start = "stmt"

In [32]:
def parse():
    """
    语法分析器
    """

    return _stmt()


def _stmt():
    root=Node(Token("stmt", None), [])
    # *stmt* -> **expr**; | **if**(**expr**) *stmt* | **for**(*optexpr*;*optexpr*;*optexpr*)*stmt* | **other**
    match (lexemes[idx]):
        case "expr":
            match(root, "expr", ";")
        case "if":
            match(root, "if", "(", "expr", ")")
            root.children.append(_stmt())
        case "for":
            match(root, "for", "(")
            _optexpr(root)
            match(root, ";")
            _optexpr(root)
            match(root, ";")
            _optexpr(root)
            match(root, ")")
            root.children.append(_stmt())
        case "other":
            match(root, "other")
        case "_":
            raise ValueError("Syntax error")
    return root


def _optexpr(root):
    # *optexpr* -> $e$ | **expr**
    if lexemes[idx] == "expr":
        match(root, "expr")


res = parse()


In [35]:
def pprint(node, indent=0):
    """
    打印语法树
    """
    print("\t" * indent + str(node.token.token_name))
    for child in node.children:
        pprint(child, indent + 1)

pprint(res)

stmt
	for
	(
	;
	expr
	;
	expr
	)
	stmt
		if
		(
		expr
		)
		stmt
			other


预测分析需要知道那些符号可能成为语法分析的一部分，这样才可以写出里面的 `switch` 语句。
在我们的例子中， FIRST 的正确计算如下所述

- FIRST(stmt) = {expr, if, for, other}
- FIRST(expr;) = {expr}

预测分析器 (predictive parser) 要求所有产生式体的 FIRST 集合不能相交。语义规则/语义动作
可以直接作为 Predictive parser 的一部分在运行过程中执行。

在我们的实现中， $e$ 就是没有任何 `match`，指针不前进。
